In [1]:
# import packages
import geopandas as gpd
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from se4g_helper import connect_right_now
import folium
from folium import plugins
import ipywidgets as widgets
from IPython.display import display

In [2]:
table_name_clmns = 'se4g_pollution_main' 

conn = connect_right_now()
cursor = conn.cursor()

db_columns = 'value_datetime_end'

# Generate the SQL statement to select data from the source table
select_data_query = f"SELECT DISTINCT {db_columns} FROM {table_name_clmns};"

# Execute the SELECT command
cursor.execute(select_data_query)

clmns = [desc[0] for desc in cursor.description]
# Fetch all the rows
rows_clmns = cursor.fetchall()

cursor.close()
conn.close()

df_columns = pd.DataFrame(rows_clmns, columns=clmns)

connected with  localhost  through psycopg2


In [3]:
def select_filters(pollutants, datetime_list):
    pollutant_dropdown = widgets.Dropdown(
        options=pollutants,
        description='Select pollutant:',
        value=pollutants[-1]
    )
    datetime_dropdown = widgets.Dropdown(
        options=datetime_list,
        description='Select datetime:',
        value=datetime_list[-1]
    )
    
    display(pollutant_dropdown)
    display(datetime_dropdown)
    
    return pollutant_dropdown, datetime_dropdown

table_name = 'se4g_pollution_main' 

conn = connect_right_now()
cursor = conn.cursor()

db_columns = 'value_numeric, samplingpoint_x, samplingpoint_y '

# Available pollutants and datetime list
pollutants = ['SO2', 'NO', 'NO2', 'CO', 'PM10']
datetime_list = df_columns['value_datetime_end'].tolist()

# Display the filter buttons
selected_pollutant, selected_datetime = select_filters(pollutants, datetime_list)

# Generate the SQL statement to select data from the source table
select_data_query = f"SELECT {db_columns} FROM {table_name} WHERE pollutant = '{selected_pollutant.value}' AND value_datetime_end = '{selected_datetime.value}';"

# Execute the SELECT command
cursor.execute(select_data_query)

columns = [desc[0] for desc in cursor.description]
#print(columns)
# Fetch all the rows
rows = cursor.fetchall()

cursor.close()
conn.close()

df = pd.DataFrame(rows, columns=columns)

geom_list = [Point(xy) for xy in zip(df['samplingpoint_x'],df['samplingpoint_y'])]
crs = 'epsg:4979'
gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geom_list)

# create a simple Folium map with markers and popups
m_folium = folium.Map(location=[57, 15], zoom_start=2.5, tiles='CartoDB positron')

# iterate through the GeoDataFrame and add a marker for each station by extracting its coordinates and the popup content
for index, row in gdf.iterrows():
    folium.Marker(
        location=[row['geometry'].y,row['geometry'].x], 
        popup=row['value_numeric'],
        icon=folium.map.Icon(color='red')
    ).add_to(m_folium)

m_heat = folium.Map(location = [55, 15], tiles='Cartodb dark_matter', zoom_start = 2.5)

heat_data = [[point.xy[1][0], point.xy[0][0]] for point in gdf.geometry]

heat_data

plugins.HeatMap(heat_data).add_to(m_heat)

#gdf.plot()

connected with  localhost  through psycopg2


Dropdown(description='Select pollutant:', index=4, options=('SO2', 'NO', 'NO2', 'CO', 'PM10'), value='PM10')

Dropdown(description='Select datetime:', index=427, options=('2023-06-09 21:00:00+01:00', '2023-06-17 09:00:00…

'\ngdf.plot() '

In [4]:
m_folium

In [5]:
m_heat